In [17]:
!pip install -q pyomo
!apt-get install -y -q glpk-utils
!pip install glpk


Reading package lists...
Building dependency tree...
Reading state information...
glpk-utils is already the newest version (5.0-1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
  Using cached glpk-0.4.8.tar.gz (160 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for glpk (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for glpk
Failed to build glpk
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (glpk)


In [28]:
!pip install mip


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 441.8/441.8 kB 29.1 MB/s eta 0:00:00
  Attempting uninstall: cffi
    Found existing installation: cffi 1.17.1
    Uninstalling cffi-1.17.1:
      Successfully uninstalled cffi-1.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pygit2 1.16.0 requires cffi>=1.17.0, but you have cffi 1.15.1 which is incompatible.


In [32]:
from gurobipy import Model, GRB, quicksum

# Cipher Parameters
num_rounds = 12  # Total number of rounds in PRINCE
state_size = 64  # State size in bits
num_sboxes = 16  # Number of S-boxes per round
sbox_size = 4    # S-box size in bits

# S-box inequalities placeholder (replace with actual PRINCE S-box constraints)
# Example: Each tuple represents a linear inequality (coefficients + constant term)
sbox_inequalities = [
    (1, -1, 0, 0, -1, 1, 0, 0, 0),  # Example inequality: x0 - x1 - y0 + y1 >= 0
    (-1, 1, 0, 0, 1, -1, 0, 0, 0),  # Replace with actual PRINCE S-box inequalities
    # Add all inequalities describing the S-box here...
]

# Correct linear layer matrix for 64-bit state
# Example: PRINCE operates on 4x4 nibbles, so expand the linear diffusion matrix
# (adjust based on the actual PRINCE diffusion matrix specification)
linear_layer_matrix = [
    # 64x64 binary matrix, representing how each output bit is computed
    # Replace with PRINCE's actual diffusion matrix
    [1 if i == j or (i + j) % state_size == 0 else 0 for j in range(state_size)]
    for i in range(state_size)
]




# Create a new model
model = Model("PRINCE_Cipher")

# Variables
X = model.addVars(num_rounds + 1, state_size, vtype=GRB.BINARY, name="X")  # State variables
K = model.addVars(num_rounds, state_size, vtype=GRB.BINARY, name="K")      # Key variables
S = model.addVars(num_rounds, num_sboxes, vtype=GRB.BINARY, name="S")      # Active S-boxes

# Objective: Minimize total active S-boxes
model.setObjective(quicksum(S[r, i] for r in range(num_rounds) for i in range(num_sboxes)), GRB.MINIMIZE)

for r in range(num_rounds):
    # Linear diffusion layer: Segment the state into 4-bit blocks
    for i in range(state_size):
        model.addConstr(
            X[r + 1, i] == quicksum(linear_layer_matrix[i][j] * X[r + 1, j] for j in range(state_size)),
            name=f"Linear_R{r}_Bit{i}"
        )

    # S-box layer: Add inequalities for each S-box
    for i in range(num_sboxes):
        sbox_input = [X[r + 1, j] for j in range(i * sbox_size, (i + 1) * sbox_size)]
        sbox_output = [X[r + 1, j] for j in range(i * sbox_size, (i + 1) * sbox_size)]  # Replace with actual output
        for inequality in sbox_inequalities:
            lhs = quicksum(coeff * var for coeff, var in zip(inequality[:-1], sbox_input + sbox_output))
            model.addConstr(lhs >= inequality[-1], name=f"Sbox_R{r}_Sbox{i}")

    # Linear diffusion layer
    for i in range(state_size):
        model.addConstr(
            X[r + 1, i] == quicksum(linear_layer_matrix[i][j] * X[r + 1, j] for j in range(state_size)),
            name=f"Linear_R{r}_Bit{i}"
        )

# Active S-box constraint: At least 16 active S-boxes in 4 consecutive rounds
window_size = 4
active_sboxes_required = 16
for r in range(num_rounds - window_size + 1):
    model.addConstr(
        quicksum(S[rr, i] for rr in range(r, r + window_size) for i in range(num_sboxes)) >= active_sboxes_required,
        name=f"ActiveSboxes_R{r}_to_R{r+window_size-1}"
    )

# Solve the model
model.optimize()

# Output results
if model.status == GRB.OPTIMAL:
    print(f"Optimal solution found with {model.objVal} active S-boxes.")
    for r in range(num_rounds):
        active_sboxes = [i for i in range(num_sboxes) if S[r, i].x > 0.5]
        print(f"Round {r}: Active S-boxes: {active_sboxes}")
else:
    print("No optimal solution found.")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 1929 rows, 1792 columns and 2064 nonzeros
Model fingerprint: 0xb9ebf3bb
Variable types: 0 continuous, 1792 integer (1792 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 2e+01]
Found heuristic solution: objective 48.0000000
Presolve removed 1929 rows and 1792 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.04 seconds (0.00 work units)
Thread count was 1 (of 2 available processors)

Solution count 1: 48 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.800000000000e+01, best bound 4.800000000000e+01, gap 0.0000%
Optimal solution found with 48.0 ac